In [17]:
#-- The current version is V5.2
#-- This is updated from v5.1 to V5.2
#-- The updated version is the improved code of the previous version.
#-- The updated version improved the month calculation to 30.4 days.
#-- Updated by Yusupha Ceesay on 02/10/2024
#-- Implemented on 03/10/2024
#-------------------------------------------------------------------------#

In [18]:
from redcap import Project
import pandas as pd
import datetime
today=pd.Timestamp.today()

#REDCap API configuration
##project configuration function
def project_configuration(api_url,api_key):
    return Project(api_url,api_key)

###indigo project configuration
indigo_project = project_configuration('https://ruff.mrc.gm:8443/redcap/api/', '63A451E2DA73C4D6A4C64A17B36B9565')

###Sensitization project configuration
sen_project = project_configuration('https://redcap.mrc.gm:8443/redcap/api/','008DBC63CA736CB68A3949DEFE43CC5D')


In [19]:
##Village mapping
village_mapping = {
'001':'Dumbuto',
'002':'Sankandi',
'003':'Nioro Jattaba',
'004':'Jattaba',
'005':'Jiffarong',
'006':'Bajana',
'007':'Kuli Kunda',
'008':'Jamaru',
'009':'Brikamanding',
'010':'Kantong Kunda',
'011':'Jali',
'013':'Manduar',
'014':'Bang Kuling',
'015':'Gissay',
'016':'Tankular',
'017':'Joli',
'018':'Kuyang',
'019':'Bantasu',
'020':'Santamba',
'021':'Missira',
'022':'Taborangkoto',
'023':'Burong',
'024':'Jula Kunda',
'025':'Karantaba',
'026':'Mandina',
'027':'Janneh Kunda',
'028':'Kemoto',
'029':'Keneba',
'030':'Batelling',
'031':'Sandeng',
'032':'Wudeba',
'034':'Kenokoto',
'035':'Manari',
'036':'Nineteen',
'040':'WUROKANG',
'041':'KWINELLA SANSANKONO',
'042':'KWINELLA NIA KUNDA',
'043':'TENDABA',
'044':'BUMARR',
'045':'BAMBAKO',
'046':'KUNDONG MARIAYA',
'047':'NEMA',
'048':'KUNDANG NUMU KUNDA',
'049':'KUNDANG FULA KUNDA',
'050':'NEMA KUTA',
'051':'JIRROFF',
'052':'MADINA ANGALLEH',
'053':'JATTA KUNDA',
'054':'MANDINA CENTRAL',
'055':'SARE SARJO',
'056':'SIBETO',
'057':'SARE NDALLA',
'058':'TABANANI',
'060':'WILLINGARA',
'061':'SARE MAMUDU'
}


In [20]:
# Function to export records from redcap and create DataFrame.
def export_records_to_df(project,forms, fields=None):
    data = project.export_records(forms=forms)
    df=pd.DataFrame(data)

    if fields:
        df=df[fields]
    return df


In [21]:
#Function to process sensitization data
def process_sensitization_data(project,village_mapping):
    fields = ['in_kiang_number', 'in_name', 'in_age', 'in_village', 'in_compound', 
        'in_eden_number', 'in_contact1', 'in_contact2', 'in_contact3', 'in_contact4', 
        'in_outcome', 'in_study_number'
        ]
    sen_log_data= export_records_to_df(project,['indigo_sensitisation'],fields)
    sen_log_data = sen_log_data[(sen_log_data['in_outcome']=='1') &
                                (sen_log_data['in_kiang_number'].notnull()) &
                                (sen_log_data['in_study_number'].notnull())
                                ]
    sen_log_data=sen_log_data.rename(columns={'in_study_number': 'con_participantid_q1'})
    sen_log_data['in_village'] = sen_log_data['in_village'].map(village_mapping)

    return sen_log_data

In [22]:
#Withdrawal data function
def process_withdrawals_data(project):
    term_data=export_records_to_df(project,['study_termination'],['con_participantid_q1','st_date','st_reason','redcap_event_name'])
    term_data['st_date']=pd.to_datetime(term_data['st_date'], errors='coerce')
    term_data=term_data[(term_data['redcap_event_name']=='end_of_study_arm_1') &
                        (term_data['st_date'].notnull())]
    
    df_Withdrawals = pd.DataFrame({
    'con_participantid_q1': term_data['con_participantid_q1'],
    'withdrawal date': term_data['st_date'],
    'Reason': term_data['st_reason'],
    'Status': 'Terminated'
    })

    # Save the DataFrame to a CSV file
    df_Withdrawals.to_csv('Withdrawals.csv', index=False)

    return df_Withdrawals


In [23]:
#FUNCTION TO PROCESS USS DATA
def process_uss_data(project):
    uss_data=export_records_to_df(project,['ultrasound_scan'], ['con_participantid_q1', 'ultsc_date', 'ultsc_type_q5', 'ultsc_reschedule', 'redcap_event_name'])
    uss_data['ultsc_date']=pd.to_datetime(uss_data['ultsc_date'], errors='coerce')
    return uss_data

In [24]:
#FUNCTION TO PROCESS WEEKLY DATA
def process_weekly_data(df,week_arm,file_name):
    weekly_df=df[(df['redcap_event_name']==week_arm)& df['ultsc_date'].notnull()]
    weekly_df=pd.DataFrame({
        'con_participantid_q1':weekly_df['con_participantid_q1'],
        'uss_date':weekly_df['ultsc_date'],
        'uss_type':weekly_df['ultsc_type_q5'],
        'Status':f'{week_arm.replace("_arm_1"," ").replace("week_","Week ")}Completed'
    })
    weekly_df.to_csv(file_name, index=False)
    return weekly_df

In [25]:
#FUNCTION TO PROCESS DELIVERY
def process_delivery_data(project):
    delv_data=export_records_to_df(project,['delivery'],['con_participantid_q1','del_vdate_q1','del_place_q2','del_lbonsetdatetime_q4', 'del_gestage', 'del_ddate_q7', 'redcap_event_name'])
    delv_data['del_ddate_q7']=pd.to_datetime(delv_data['del_ddate_q7'], errors='coerce')

    ##conditions for delivery to be counted.
    delv_data=delv_data[(delv_data['redcap_event_name']=='delivery__postpart_arm_1') &
                        (delv_data['del_ddate_q7'].notnull())]

    ##create dataframe for delivery
    delivery_df = pd.DataFrame({
        'con_participantid_q1': delv_data['con_participantid_q1'],
        'del_ddate_q7': delv_data['del_ddate_q7'],
        'GA at Delivery': delv_data['del_gestage'],
        'Status': 'Delivered'
    })
    
    ## Save the DataFrame to a CSV file
    delivery_df.to_csv('Delivery.csv', index=False)
    return delivery_df



In [26]:
#FUNCTION TO PROCESS INFANT DATA
def process_infant_data(project):
    ##extract infant anthrops and stool
    inf_data=export_records_to_df(project,['infant_anthropometry','infant_stool_collection'],['con_participantid_q1','ina_infantid','ian_date','ian_visitoutcome','infant_anthropometry_complete','isc_dvdate_q1','isc_dtime_q2','isc_stoolcollected_q3','infant_stool_collection_complete','redcap_event_name'])

    #dates as a date data type
    inf_data['ian_date'] = pd.to_datetime(inf_data['ian_date'], errors='coerce')
    inf_data['isc_dvdate_q1'] = pd.to_datetime(inf_data['isc_dvdate_q1'], errors='coerce')
    inf_data['isc_dtime_q2'] = pd.to_datetime(inf_data['isc_dtime_q2'], errors='coerce').dt.time

    ##apply conditions to infant data
    inf_data[
            ((inf_data['ian_date'].notnull())) &
            ((inf_data['ian_visitoutcome']=='0') | (inf_data['ian_visitoutcome']=='1')) &
            (inf_data['infant_anthropometry_complete']=='2') &
            ((inf_data['isc_dvdate_q1'].notnull())) &
            ((inf_data['isc_stoolcollected_q3']=='0') | (inf_data['isc_stoolcollected_q3']=='1')) &
            (inf_data['infant_stool_collection_complete']=='2')
 ]
    return inf_data
    


In [27]:
#FUNCTION TO PROCESS INFANT MONTHLY DATA
def process_month_data(infant_df,df_delv,month_name):
    inf_month=infant_df[(infant_df['redcap_event_name']==month_name)]
    
    ##merge to delivery data based on participant id
    month_infant_mother=df_delv.merge(inf_month, on='con_participantid_q1', how='left')

    # Return required variables based on common participant ID
    month_infant_mother = month_infant_mother[['con_participantid_q1','del_ddate_q7','ina_infantid','ian_date','isc_stoolcollected_q3']]
    month_infant_mother=month_infant_mother[
        ((month_infant_mother['del_ddate_q7'].notnull())) &
        ((month_infant_mother['ian_date'].notnull())) &
        ((month_infant_mother['isc_stoolcollected_q3']=='0') | (month_infant_mother['isc_stoolcollected_q3']=='1'))
        
    ]

    ##create dataframe for monthly visits
    month_df = pd.DataFrame({
        'con_participantid_q1': month_infant_mother['con_participantid_q1'],
        'Delivery Date': month_infant_mother['del_ddate_q7'],
        'Infant ID': month_infant_mother['ina_infantid'],
        'Infant anthrops Date': month_infant_mother['ian_date'],
        'Status': f'{month_name.replace("_arm_2", " ").replace("month_", "Month ")}Completed'
    })

    month_df.to_csv(f'{month_name.replace("_arm_2","").replace("month_","Month ")}.csv', index=False)
    return month_df

USE FUNCTIONS TO GENERATE DATA

In [28]:
#SENSITIZATION DATA
sen_log_data=process_sensitization_data(sen_project,village_mapping)

#WITHDRAWAL DATA
df_withdrawals=process_withdrawals_data(indigo_project)

#USS DATA
df_uss=process_uss_data(indigo_project)

In [29]:
#WEEKLY ANTENATAL DATA
wk20_df=process_weekly_data(df_uss,'week_20_arm_1','Week 20.csv')
wk28_df=process_weekly_data(df_uss,'week_28_arm_1','Week 28.csv')
wk36_df=process_weekly_data(df_uss,'week_36_arm_1','Week 36.csv')

In [30]:
#DELIVERY DATA
df_delivery=process_delivery_data(indigo_project)

#INFANT DATA
df_infant=process_infant_data(indigo_project)

In [31]:
#PROCESS DATA FOR EACH MONTH
month_names=[
    'month_1_arm_2', 'month_2_arm_2', 'month_3_arm_2', 'month_4_arm_2',
    'month_5_arm_2', 'month_6_arm_2', 'month_7_arm_2', 'month_8_arm_2',
    'month_9_arm_2', 'month_10_arm_2', 'month_11_arm_2', 'month_12_arm_2'

]

##process and save data for each month in a for loop
for month in month_names:
    process_month_data(df_infant,df_delivery,month)

ANTENATAL CALLS

In [32]:

def load_data(file_path):
    return pd.read_csv(file_path)

def merge_and_filter(df_main, df_filter, key):
    merged_df = pd.merge(df_main, df_filter, on=key, how='inner')
    return df_main[~df_main[key].isin(merged_df[key])]

def process_data(df, visit_type, gestational_weeks_limit=None):
    df = df[['con_participantid_q1', 'con_participant_eden_num_q11', 'ultsc_date', 'ultsc_reschedule',
             'ga_weeks', 'ga_days', 'con_cluster_q12', 'redcap_event_name']].copy()
    
    if visit_type == 'Rescheduled':
        df = df[(df['redcap_event_name'] == 'recruitment_arm_1') &
                (df['ultsc_date'].notnull()) &
                (df['ultsc_reschedule'].notnull())]
    elif gestational_weeks_limit:
        df['ga_weeks'] = pd.to_numeric(df['ga_weeks'], errors='coerce')
        df['ga_days'] = pd.to_numeric(df['ga_days'], errors='coerce')
        df = df[(df['redcap_event_name'] == 'recruitment_arm_1') &
                (df['ultsc_date'].notnull()) &
                (df['ga_weeks'] <= gestational_weeks_limit)]
    else:
        df = df[(df['redcap_event_name'] == 'recruitment_arm_1') &
                (df['ultsc_date'].notnull()) &
                (df['ultsc_reschedule'].notnull())]
    
    return df

def calculate_dates(df, visit_type):
    today = pd.Timestamp.today()
    
    df['uss_date'] = pd.to_datetime(df['ultsc_date'])
    df['due_date'] = pd.to_datetime(df['ultsc_reschedule'])
    
    if visit_type != 'Rescheduled':
        df['ga_weeks'] = pd.to_numeric(df['ga_weeks'], errors='coerce')
        df['ga_days'] = pd.to_numeric(df['ga_days'], errors='coerce')
        
        total_ga_days = (df['ga_weeks'] * 7) + df['ga_days']
        df['edd'] = df['uss_date'] + pd.to_timedelta(280 - total_ga_days, unit='days')
        
        days_diff = (today - df['uss_date']).dt.days
        df['current_ga'] = (days_diff + total_ga_days) / 7
        
        visit_dates = {
            'Week 20': (140, 147),
            'Week 28': (196, 203),
            'Week 36': (252, 259)
        }
        
        if visit_type in visit_dates:
            start_diff, end_diff = visit_dates[visit_type]
            df['early_date'] = df['uss_date'] + pd.to_timedelta((start_diff - 3) - total_ga_days, unit='days')
            df['due_date'] = df['uss_date'] + pd.to_timedelta(start_diff - total_ga_days, unit='days')
            df['last_date'] = df['uss_date'] + pd.to_timedelta(end_diff - total_ga_days, unit='days')
    else:
        df['current_ga'] = None
        df['edd'] = None
        df['last_date'] = df['due_date']
        df['early_date']=df['due_date'] - pd.to_timedelta(3, unit='days')
    
    return df

def create_dataframe(df, visit_type):
    # Ensure all required columns are present
    required_columns = ['con_participantid_q1', 'con_cluster_q12', 'uss_date', 'con_participant_eden_num_q11', 
                        'ga_weeks', 'ga_days', 'edd', 'current_ga', 'early_date','due_date', 'last_date']
    
    for col in required_columns:
        if col not in df.columns:
            df[col] = None
    
    df_processed = df[required_columns].copy()
    df_processed.columns = ['con_participantid_q1', 'Cluster', 'USS Date', 'EDEN | Infant Number', 
                            'Gestational Age (Weeks)', 'Gestational Age (Days)', 
                            'EDD | Delivery date', 'Current Gestational Age','Early Date',
                            'Due Date', 'Last Date']
    df_processed['Visit'] = visit_type
    return df_processed

def main():
    # Load project data
    project_data = indigo_project.export_records(forms=['consent', 'ultrasound_scan'])
    project_df = pd.DataFrame(project_data)

    # Rescheduled
    rescheduled_data = process_data(project_df, visit_type='Rescheduled')
    rescheduled_data = merge_and_filter(rescheduled_data, load_data('Withdrawals.csv'), 'con_participantid_q1')
    rescheduled_data = merge_and_filter(rescheduled_data, load_data('Delivery.csv'), 'con_participantid_q1')
    rescheduled_df = calculate_dates(rescheduled_data, visit_type='Rescheduled')
    rescheduled_df = create_dataframe(rescheduled_df, visit_type='Rescheduled')
    
    # Week 20
    week_20_data = process_data(project_df, visit_type='Week 20', gestational_weeks_limit=20)
    week_20_data = merge_and_filter(week_20_data, load_data('Withdrawals.csv'), 'con_participantid_q1')
    week_20_data = merge_and_filter(week_20_data, load_data('Delivery.csv'), 'con_participantid_q1')
    week_20_df = calculate_dates(week_20_data, visit_type='Week 20')
    week_20_df = create_dataframe(week_20_df, visit_type='Week 20')
    week_20_df = merge_and_filter(week_20_df, load_data('Week 20.csv'), 'con_participantid_q1')
    
    # Week 28
    week_28_data = process_data(project_df, visit_type='Week 28')
    week_28_data = merge_and_filter(week_28_data, load_data('Withdrawals.csv'), 'con_participantid_q1')
    week_28_data = merge_and_filter(week_28_data, load_data('Delivery.csv'), 'con_participantid_q1')
    week_28_df = calculate_dates(week_28_data, visit_type='Week 28')
    week_28_df = create_dataframe(week_28_df, visit_type='Week 28')
    week_28_df = merge_and_filter(week_28_df, load_data('Week 28.csv'), 'con_participantid_q1')
    
    # Week 36
    week_36_data = process_data(project_df, visit_type='Week 36')
    week_36_data = merge_and_filter(week_36_data, load_data('Withdrawals.csv'), 'con_participantid_q1')
    week_36_data = merge_and_filter(week_36_data, load_data('Delivery.csv'), 'con_participantid_q1')
    week_36_df = calculate_dates(week_36_data, visit_type='Week 36')
    week_36_df = create_dataframe(week_36_df, visit_type='Week 36')
    week_36_df = merge_and_filter(week_36_df, load_data('Week 36.csv'), 'con_participantid_q1')
    
    # Combine all DataFrames
    pregnancy_call_df = pd.concat([rescheduled_df, week_20_df, week_28_df, week_36_df], ignore_index=True)
    return pregnancy_call_df
# Run the main function and get the combined DataFrame
antenatal_df = main()


Post-natal calls

In [33]:
#Function to read and filter data based on exclusion lists
def read_and_exclude(filename,df,key):
    exclusion_df=pd.read_csv(filename)
    merged_df=pd.merge(df,exclusion_df,on=key,how='inner')
    return df[~df[key].isin(merged_df[key])]

In [34]:
# #Function to calculate due date and last date
# def calculate_infant_dates(df, months, days=None):
#     df['due_date'] = df['del_ddate_q7'] + pd.DateOffset(months=months)
#     df['early_date'] = df['due_date'] - pd.DateOffset (days= 3)
#     if days is not None:
#         df['last_date'] = df['del_ddate_q7'] + pd.DateOffset(months=months, days=days)
#     else:
#         df['last_date'] = df['del_ddate_q7'] + pd.DateOffset(months=months)
#     return df


In [35]:

def calculate_infant_dates(df, months, win_days=None):
    # Convert months to days assuming 30.4 days per month
    total_days = 30.4 * months
    
    df['due_date'] = df['del_ddate_q7'] + pd.DateOffset(days=total_days)
    df['early_date'] = df['due_date'] - pd.DateOffset(days=3)
    if win_days is not None:
        df['last_date'] = df['del_ddate_q7'] + pd.DateOffset(days=total_days + win_days)
    else:
        df['last_date'] = df['del_ddate_q7'] + pd.DateOffset(days=total_days)
    
    df['due_date'] = df['due_date'].dt.date
    df['early_date'] = df['early_date'].dt.date
    df['last_date'] = df['last_date'].dt.date
    
    return df


In [36]:
# Post Partum Call
def post_partum_call(project, df_delv):
    # Extract neonate screening and infant ID
    neonate_df=export_records_to_df(project,['infantid', 'neonate_screening'],['con_participantid_q1', 'infantid', 'neo_vdate', 'redcap_event_name'])
    neonate_df = neonate_df[
        (neonate_df['redcap_event_name'] == 'baby_check_arm_2') &
        (neonate_df['neo_vdate'].notnull())
    ]

    ##merge neonate data with delivery
    neonate_mother = df_delv.merge(neonate_df, on='con_participantid_q1', how='inner')
    neonate_mother = neonate_mother[['con_participantid_q1', 'del_ddate_q7', 'infantid']]

    ## Exclude withdrawals
    neonate_mother = read_and_exclude('Withdrawals.csv', neonate_mother, 'con_participantid_q1')

    return neonate_mother


In [37]:
# Month Calls
def infant_month_call(neonate_mother, months, visit_name, days=None):
    neonate_mother = pd.DataFrame(neonate_mother)
    neonate_mother['del_ddate_q7'] = pd.to_datetime(neonate_mother['del_ddate_q7'])
    neonate_mother = calculate_infant_dates(neonate_mother, months, days)
    
    month_df = pd.DataFrame({
        'con_participantid_q1': neonate_mother['con_participantid_q1'],
        'EDD | Delivery date': pd.to_datetime(neonate_mother['del_ddate_q7']),
        'EDEN | Infant Number': neonate_mother['infantid'],
        'Early Date' : neonate_mother['early_date'],
        'Due Date': neonate_mother['due_date'],
        'Last Date': neonate_mother['last_date'],
        'Visit': visit_name
    })

    ## Exclude existing data
    month_data = pd.read_csv(f'{visit_name}.csv')
    month_df = read_and_exclude(f'{visit_name}.csv', month_df, 'con_participantid_q1')
    
    return month_df


In [38]:
def post_natal():
    # Post Partum Call
    neonate_mother = post_partum_call(indigo_project, df_delivery)

    # Month Calls
    month1_df = infant_month_call(neonate_mother, 1, 'Month 1', days=14)
    month2_df = infant_month_call(neonate_mother, 2, 'Month 2', days=7)
    month3_df = infant_month_call(neonate_mother, 3, 'Month 3', days=7)
    month4_df = infant_month_call(neonate_mother, 4, 'Month 4', days=7)
    month5_df = infant_month_call(neonate_mother, 5, 'Month 5', days=7)
    month6_df = infant_month_call(neonate_mother, 6, 'Month 6', days=14)
    month7_df = infant_month_call(neonate_mother, 7, 'Month 7', days=7)
    month8_df = infant_month_call(neonate_mother, 8, 'Month 8', days=7)
    month9_df = infant_month_call(neonate_mother, 9, 'Month 9', days=7)
    month10_df = infant_month_call(neonate_mother, 10, 'Month 10', days=7)
    month11_df = infant_month_call(neonate_mother, 11, 'Month 11', days=7)
    month12_df = infant_month_call(neonate_mother, 12, 'Month 12', days=14)

    infant_month_df = pd.concat([month1_df,month2_df,month3_df,month4_df,month5_df,month6_df,month7_df,month8_df,month9_df,month10_df,month11_df,month12_df], ignore_index=True)
    return infant_month_df

In [39]:
postpartum_df=post_natal()

Final Call-List
##concatinate infant and maternal call list

In [40]:
#Function to concatinate thedataframes
def concatenate_due_dates(*dfs):
    return pd.concat(dfs, ignore_index=True)


In [41]:
# Function to merge with sensitization data
def merge_with_sensitization(due_dates_df, sensitization_df):
    return pd.merge(due_dates_df, sensitization_df, on='con_participantid_q1', how='left')


In [42]:
# Function to format final DataFrame
def format_final_df(df):
    return pd.DataFrame({
        'PARTICIPANT ID': df['con_participantid_q1'],
        'EDEN | INFANT ID': df['EDEN | Infant Number'],
        'KIANG NUMBER': df['in_kiang_number'],
        'NAME': df['in_name'],
        'CURRENT GA': df['Current Gestational Age'],
        '(Expected)Delivery Date ((E)DD)': pd.to_datetime(df['EDD | Delivery date']),
        'VILLAGE': df['in_village'],
        'COMPOUND': df['in_compound'],
        'CONTACT1': df['in_contact1'],
        'CONTACT2': df['in_contact2'],
        'CONTACT3': df['in_contact3'],
        'CONTACT4': df['in_contact4'],
        'EARLY DATE':pd.to_datetime(df['Early Date']),
        'DUE DATE': pd.to_datetime(df['Due Date']),
        'LAST DUE DATE':pd.to_datetime( df['Last Date']),
        'Visit': df['Visit']
    })

In [43]:

# Function to calculate the target due date
def calculate_target_due_date():
    current_date = datetime.date.today()
    next_monday = current_date + datetime.timedelta(days=(0 - current_date.weekday() + 7) % 7)
    target_due_date = next_monday + datetime.timedelta(days=3)
    return pd.to_datetime(target_due_date)

# Function to filter call list based on due date
def filter_call_list(df_due_dates, target_due_date):
    return df_due_dates[df_due_dates['EARLY DATE'] <= target_due_date]


In [44]:
# Function to save DataFrame to CSV with a timestamp
def save_to_csv(df, base_filename):
    now = datetime.datetime.now()
    timestamp = now.strftime('%Y%m%d_%H%M%S')
    filename = f'{base_filename}_{timestamp}.csv'
    df.to_csv(filename, index=False)


In [45]:
# Generate final due dates DataFrame
df_due_dates1 = concatenate_due_dates(antenatal_df,postpartum_df)


In [46]:
# Merge with sensitization data
df_due_dates = merge_with_sensitization(df_due_dates1, sen_log_data)

# Format the final DataFrame
df_due_dates_formatted = format_final_df(df_due_dates)

# Calculate the target due date
target_due_date = calculate_target_due_date()


c:\Users\yuceesay\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\algorithms.py:431: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  uniques = table.unique(values)
c:\Users\yuceesay\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexes\base.py:2023: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return self._engine.is_unique
c:\Users\yuceesay\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexes\base.py:3512: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. 

In [47]:
# Filter the call list
df_call_list = filter_call_list(df_due_dates_formatted, target_due_date)

# Save the call list to a CSV file
save_to_csv(df_call_list, 'CallList_v5.2')


In [48]:


def process_labels(df):
    results = []
    for index, row in df.iterrows():

        maternal_visit= "Maternal "+"(" + row['Visit'] + ")"
        infant_visit = "Infant "+"(" + row['Visit'] + ")"
        leo_number = "LEO25071"
        #Samples
        #edta = "EDTA"
        #lithHep = "LithHep"
        breastmilk = "BREAST MILK"
        #urine = "URINE"
        stool="STOOL"
        partID= row['PARTICIPANT ID']
        infID= row['EDEN | INFANT ID']
        due_date = row['EARLY DATE']

        if row['Visit'] == 'Month 1':
            results.append([due_date, partID, maternal_visit, leo_number, breastmilk])
            results.append([due_date, partID, maternal_visit, leo_number, breastmilk])
            #results.append([due_date, infID, infant_visit, leo_number, edta])
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 2':
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 3':
            results.append([due_date, partID, maternal_visit, leo_number, breastmilk])
            results.append([due_date, partID, maternal_visit, leo_number, breastmilk])
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 4':
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 5':
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 6':
            results.append([due_date, partID, maternal_visit, leo_number, breastmilk])
            results.append([due_date, partID, maternal_visit, leo_number, breastmilk])
            #results.append([due_date, infID, infant_visit, leo_number, edta])
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 7':
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 8':
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 9':
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 10':
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 11':
            results.append([due_date, infID, infant_visit, leo_number, stool])
        elif row['Visit'] == 'Month 12':
            results.append([due_date, partID, maternal_visit, leo_number, breastmilk])
            results.append([due_date, partID, maternal_visit, leo_number, breastmilk])
            #results.append([due_date, infID, infant_visit, leo_number, edta])
            results.append([due_date, infID, infant_visit, leo_number, stool])   

    return pd.DataFrame(results, columns=['EARLY DATE','ID', 'Visit', 'LEO Number', 'Sample'])




In [49]:
# Process data
df_labels = process_labels(df_call_list)

# Save the to a CSV file
save_to_csv(df_labels, 'INDiGO Labels_v1.2')